---
title: Week 5 Assignment, CO2 in Soda Water
subtitle: Orchestra Scenario, CO2 in a Soda bottle
author:
  - name: Timo Heimovaara
    affiliations: Delft University of Technology, department of Geoscience & Engineering
    orcid: 
    email: t.j.heimovaara@tudelft.nl
license: CC-BY-NC-ND-4.0 (https://creativecommons.org/licenses/by-nc-nd/4.0/).
date: 2026-01-12
kernelspec:
    name: python3
    display_name: 'Python 3.13'
---

## Assignment
In this assignment you will analyse the CO2 - water equilibrium in a Soda-water bottle.

We will use Orchestra to check how the system behaves when we work with a fixed gas volume and when we work with a fixed pressure

In [1]:
# Import libraries required for running all simulations
import os
import sys
from IPython.utils import capture
from IPython.display import display, Markdown
from pathlib import Path
from contextlib import chdir

import numpy as np
import matplotlib.pyplot as plt
import PyORCHESTRA # here, the ORCHESTRA submodule is imported
import pandas as pd
import seaborn as sns

%matplotlib widget
sns.set()

# Prepare a file to capture PyOrchestra output
capture_file = open("pyorchestra_output.log", "w")


# pyOrchestra is implemented in C++
# Save original stdout file descriptor
original_stdout_fd = sys.stdout.fileno()

# Duplicate original stdout so we can restore it later
saved_stdout_fd = os.dup(original_stdout_fd)



# We need to import some Orchestra files. We need to know the path layout on 
# the local machine:
def find_book_root(start: Path | None = None) -> Path:
    """
    Walk upward from `start` (or CWD) until a directory containing Jupyter Book
    marker files is found. Returns the path to the book root.
    Raises FileNotFoundError if no root is found.
    """
    config_any = {"_config.yml", "_config.yaml"}      # some projectrs use .yaml
    myst_any = {"myst.yml", "myst.yaml"}            # jupyter-book uses _toc.yml

    cur = Path(Path.cwd()).resolve()

    for parent in [cur, *cur.parents]:
        children = {f.name for f in parent.iterdir()} if parent.exists() else set()
        has_any_config = bool(config_any & children)
        has_any_myst = bool(myst_any & children)
        if has_any_config and has_any_myst:
            return parent

    raise FileNotFoundError(
        f"Could not find Jupyter Book root (no _config.y* and myst.y* found above {cur})"
    )


def path_from_book_root(*parts: str | Path) -> Path:
    root = find_book_root()
    p = (root.joinpath(*parts)).resolve()
    if not p.exists():
        raise FileNotFoundError(f"Path not found: {p}")
    return p

# In order for orchestra to run we need to change directory to the directory with the input file:


# Example usage:
# input_file = path_from_book_root("content", "week 02", "Orchestra_simulation", "chemistry1.inp")
# print("Input file:", input_file)
   
orchestra_path = path_from_book_root("content", "week 05", "Assignment_pressure_volume")
# print(orchestra_path)

## Download this script and the required ORCHESTRA files
TODO Create an yml for a local python environment
TODO Create zip file with the correct files. Explain how to install Orchestra using pip in a local python environment.
TODO Update all files so that they are well documented and describe the system

pyorchestra will be initialize with the correct chemistry file...

explain that InVars are the species used to define the totals. 
explain that OutVars is the list of Orchestra variables that are exported to python for post-processing or necessary for simulation.

Once prinicple problem is known we can create and initialize a pyorchestra object.

## Define chemical system: Carbonate, Water and Calcite using the ORCHESTRA-GUI
In order to solve a chemical equilibrium problem with pyOrchestra, we need to define our chemical system first. Orchestra defines this system with a number of text files. The most important one is the so-called chemistry input file (*chemistry1.inp*). This file is most easily made using the Orchestra GUI which can be accessed by clicking on the orchestra2023.jar file.

For the CO2-water system we distinguish three types of scenarios: 
1. a scenario with a gas-phase with a fixed closed volume. This implies that the pressure in the gas phase will change based on the chemical processes occuring in the system;
2. a scenario where the gas pressure is assumed to be constant in a closed volume, this implies that the volume of the system changes based on the chemical processes;
3. a scenario where our system is in open contact with the atmosphere, this implies that the partial pressures (and therefore the activities) of the gaseous components do not change as the reactions proceed in our system. This because the volume (and therefore the amount of gaseous species present) is so much larger than our system that we may neglect the changes due to our reactions.

Similar to the first tutorial of this week we start with a skeleton problem (a working ORCHESTRA simulation and it is convenient to start with the results from tutorial 1) and then adjust the files step by step. 

Using the steps described in Tutorial 1 we created *chemistry_pressure_volume.inp*. The carbonate system we want to analyse is controlled by the following reactions:
$$
\begin{aligned}
\text{CO}_2\text{(g)} + \text{H}_2\text{O} &\leftrightharpoons \text{H}_2\text{CO}_3^* \\
\text{H}_2\text{CO}_3^* &\leftrightharpoons \text{H}^+ + \text{HCO}_3^- \\
\text{HCO}_3^- &\leftrightharpoons \text{H}^+ + \text{CO}_3^{-2} \\
\text{H}_2\text{O} &\leftrightharpoons \text{H}^+ + \text{OH}^- \\
\text{Ar(g)} + \text{H}_2\text{O} &\leftrightharpoons \text{Ar(aq)} \\
\end {aligned}
$$


$\text{Ar(g)}$ is used as an inert back ground gas to maintain gas pressure in the case all of the gaseous species in our reaction were to be consumed. 

Using all insights from above we generated the following master species table: 

|Primary entity|Input Variable|Fix log activity|Log activity|Concentration|Phase|Expression|
|--|--|--|--|--|--|--|
|Ar[g]|||1.0e-9||tot||
|CO2[g]||||1.0e-9|tot||
|H|pH|x|7.0|||H.logact=-pH|
|H2O||x|-0.0||||

Orchestra will use these primary entities, and their initial values to calculate the total elemental composition in the complete system. 

We have another *chemistry.inp* file in order to calulate the equilibrium when we have a "open system" where the water is in contact with a gas phase where the partial pressures are maintained at a constant value. This is the case for a jar open to the atmosphere or a jar in contact with a control system that maintains a constant partial pressure.
```{exercise} 
What is the reason why we may assume that the partial pressures are constant when a jar of water is open to the atmosphere?
```
Fixing the log-activties during simulation allows the totals of the compounds to vary during the simulation, as long as the log-activity condition is maintained. 

Please note that we allow ORCHESTRA to change the value of the pH to manage the charge balance.

### Phases & Reactions
On the **Phases & Reactions** tab shown in figure [](#phases_reactions) we define the reaction network of our system. Please ensure that all reactions are marked.

### Other tabs
There is no need to change anything on the other tabs as these are correctly set in Tutorial 1.

### Using Orchestra to run the scenario
It is possible to use the ORCHESTRA GUI to run the simulation, and this has been prepared using the **Input** and **Output** tabs on the right of the window. Please have look at these tabs and they should be rather "self explanatory". 



## Running the CO2 in a Soda Bottle scenario in a Python notebook
@@@@
We follow the same systematic approach as Tutorial 1, which consists of the following standard steps:
1. define the domain for our problem
2. define the primary species which change during our scenario
3. initialize the problem
4. run the problem
5. process the output

### 1. Define the domain
For this analysis you need to define if the problem is a fixed_pressure, fixed_volume or constant log_activity.
You also need to define the volumes of gas a water present.

### 2. Define the primary species
The primary species are as defined above with the ORCHESTRA GUI. Please note that the values required by ORCHESTRA have to be in moles. In order to get moles/liter the volume of water has to be set to 1 liter, and the density of water need to be given as well.

### 3. Initialise the problem 
The initial condition for our problem will be defined by setting:
1. the type of problem (fixed logactivty, volume or pressure)
2. the initial conditions (volumes and totals of gas present)

Once pyOrchestra is initialized, running a simulation consists of a series of steps where the values of the required set of input variables are passed via *InVARS* to ORCHESTRA, after which a set of corresponding output variables are passed back in *OutVars*. Please note that the type of problem has to be defined as well.

After initialization in Python, we know which variables will be passed through *OutVars* and can be used in *InVars*.

The following code shows how to do this.
```{note} 
It is relatively easy to change the OutVars array. In order to find all possible variables in ORCHESTRA, check the **Phases & Reactions** tab on the **Chemistry** tab in the ORCHESTRA-GUI .
```


In [2]:
#--- Initialize Pressure Volume problem --- 
# for initialization we need to temporarily move to the directory containing the 'chemistry1.inp' file.
with chdir(orchestra_path):

    # Input file is generated with Orchestra GUI
    InputFile = 'chemistry_pressure_volume.inp'
    NoCells = 1 #only 1 cell to have a 0-D system with 1liter of water
    
    # We define the input variables that will be changed in the script
    # Please note that gas_type indicates type of problem:
    #  fixed-gasvolume: gas_type = 0  --> pressure to be calculated
    #  fixed-pressure: gas_type = 1  --> gasvolume to be calculated
    
    InVars_pv = np.array(['Ar[g].tot', 
                       'CO2[g].tot',
                       'watervolume',
                       'gasvolume_fixed',
                       'pressure_fixed',
                       'gas_type'
                       ])
    
    # We select the output from Orchestra we need to use
    OutVars_pv = np.array(['pH', 'H+.con', 'H+.tot', 'H+.logact',
                        'OH-.tot', 'OH-.con', 'OH-.logact',
                        'CO2[g].tot', 'CO2[g].con', 'CO2[g].diss', 'CO2[g].logact',
                        'CO3-2.tot', 'CO3-2.con', 'CO3-2.logact','CO3-2.diss',
                        'HCO3-.tot', 'HCO3-.con', 'HCO3-.logact', 'HCO3-.diss',
                        'H2CO3.tot', 'H2CO3.con', 'H2CO3.logact', 'H2CO3.diss',
                        'Ar[g].tot', 'Ar[g].con', 'Ar[g].diss', 'Ar[g].logact', 
                        'I', 'chargebalance', 'watervolume', 'gasvolume','pressure',
                        'gas_type', 'pressure_fixed', 'gasvolume_fixed'
                    ])

    # Associate a variable with the pyOrchestra.ORCHESTRA() class
    p_pressure_volume = PyORCHESTRA.ORCHESTRA()

    # Initialize the class with the parameters defined above
    p_pressure_volume.initialise(InputFile, NoCells, InVars_pv, OutVars_pv)


Reading and expanding calculator new stylechemistry_pressure_volume.inp
Scanning file: chemistry_pressure_volume.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
Scanning file: chemistry_pressure_volume.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
Including file: chemistry_pressure_volume.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
0.02 sec.
	Reading variables .... 0.005 s
testing:
8:Ar[g].tot
10:CO2[g].tot
13:watervolume
14:gasvolume_fixed
15:pressure_fixed
16:gas_type
11:pH
17:H+.con
18:H+.tot
19:H+.logact
20:OH-.tot
21:OH-.con
22:OH-.logact
10:CO2[g].tot
23:CO2[g].con
24:CO2[g].diss
9:CO2[g].logact
25:CO3-2.tot
26:CO3-2.con
27:CO3-2.logact
28:CO3-2.diss
29:HCO3-.tot
30:HCO3-.con
31:HCO3-.logact
32:HCO3-.diss
33:H2CO3.tot
34:H2CO3.con
35:H2CO3.logact
36:H2CO3.diss
8:Ar[g].tot
37:Ar[g].con
38:Ar[g].diss
7:Ar[g].logact
3:I
6:chargeb

In [3]:
#--- Initialize Fixed Logact problem --- 
# for initialization we need to temporarily move to the directory containing the 'chemistry1.inp' file.
with chdir(orchestra_path):

    # Input file is generated with Orchestra GUI
    InputFile = 'chemistry_fixed_logact.inp'
    NoCells = 1 #only 1 cell to have a 0-D system with 1liter of water
    
    # We define the input variables that will be changed in the script
    InVars_fl = np.array(['CO2[g].logact',
                       'Ar[g].logact', 
                       'watervolume',
                       'gasvolume_fixed',
                       'pressure_fixed', # this is not used... as the pressure comes from logact!  
                       'gas_type'
                       ])
    
    # We select the output from Orchestra we need to use
    OutVars_fl = np.array(['pH', 'H+.con', 'H+.tot', 'H+.logact',
                        'OH-.tot', 'OH-.con', 'OH-.logact',
                        'CO2[g].tot', 'CO2[g].con', 'CO2[g].diss', 'CO2[g].logact',
                        'CO3-2.tot', 'CO3-2.con', 'CO3-2.logact','CO3-2.diss',
                        'HCO3-.tot', 'HCO3-.con', 'HCO3-.logact', 'HCO3-.diss',
                        'H2CO3.tot', 'H2CO3.con', 'H2CO3.logact', 'H2CO3.diss',
                        'Ar[g].tot', 'Ar[g].con', 'Ar[g].diss', 'Ar[g].logact', 
                        'I', 'chargebalance', 'watervolume', 'gasvolume','pressure','gas_type',
                        'gasvolume_fixed', 'pressure_fixed'
                        ])

    # Associate a variable with the pyOrchestra.ORCHESTRA() class
    p_fixed_logact = PyORCHESTRA.ORCHESTRA()

    # Initialize the class with the parameters defined above
    p_fixed_logact.initialise(InputFile, NoCells, InVars_fl, OutVars_fl)


Reading and expanding calculator new stylechemistry_fixed_logact.inp
Scanning file: chemistry_fixed_logact.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
Scanning file: chemistry_fixed_logact.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
Including file: chemistry_fixed_logact.inp
Scanning file: objects2025.txt
Scanning file: kinetics_db.txt
Including file: objects2025.txt
0.023 sec.
	Reading variables .... 0.005 s
testing:
8:CO2[g].logact
7:Ar[g].logact
10:watervolume
11:gasvolume_fixed
12:pressure_fixed
13:gas_type
9:pH
14:H+.con
15:H+.tot
16:H+.logact
17:OH-.tot
18:OH-.con
19:OH-.logact
20:CO2[g].tot
21:CO2[g].con
22:CO2[g].diss
8:CO2[g].logact
23:CO3-2.tot
24:CO3-2.con
25:CO3-2.logact
26:CO3-2.diss
27:HCO3-.tot
28:HCO3-.con
29:HCO3-.logact
30:HCO3-.diss
31:H2CO3.tot
32:H2CO3.con
33:H2CO3.logact
34:H2CO3.diss
35:Ar[g].tot
36:Ar[g].con
37:Ar[g].diss
7:Ar[g].logact
3:I
6:chargebalance

Please note that the output of this code is what ORCHESTRA echos back. ORCHESTRA uses a set of variables in order to store the input variables and the results of the calculations, in this case 33. The top part of the output shows the output requested by us through *OutVars* together with the values used during initialization.

### Run the Problem
Here we illustrate how to run this problem, we demonstrate all three possible scenarios:
1. fixed gas volume
2. fixed pressure
3. fixed logactivity of gaseous species

### Scenario 1: fixed gas volume

In [4]:
# %%
# Scenario 1, fixed gasvolume
# InVars_pv need to contain floats
IN = np.array([np.ones_like(InVars_pv)]).astype(float)

IN[0][np.where(InVars_pv == 'gas_type')] = 0 # fixed gasvolume = 0, fixed pressure = 1
IN[0][np.where(InVars_pv == 'CO2[g].tot')] = 1e-10 # moles virtually no CO2 in system
IN[0][np.where(InVars_pv == 'Ar[g].tot')] = 4.1e-2 # moles for about 1 atm Argon pressure (estimated from ideal gas law)
IN[0][np.where(InVars_pv == 'watervolume')] = 1.0 # 1 liter
IN[0][np.where(InVars_pv == 'gasvolume_fixed')] = 1.0 # 1 liter
IN[0][np.where(InVars_pv == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_pressure_volume.set_and_calculate(IN)
Res_Scenario1 = pd.DataFrame(OUT,columns=OutVars_pv)
print(Res_Scenario1.T)

Try a first calculation with iia switched off....
Parsing expressions of chemistry_pressure_volume.inp..... 
Optimizing expressions of chemistry_pressure_volume.inp..... 0.03 sec.
899 variables, 2117 expressions, 3 equations.
First calculation was successful!
Repeat calculation with iia switched on..
Switching on: logI: -2
Switching on: pH: 7
This was successful!!
                            0
pH               7.048166e+00
H+.con           8.953376e-08
H+.tot           8.951963e-08
H+.logact       -7.048166e+00
OH-.tot          8.951963e-08
OH-.con          8.951963e-08
OH-.logact      -7.048234e+00
CO2[g].tot       1.000000e-10
CO2[g].con       8.303994e-11
CO2[g].diss      1.696006e-11
CO2[g].logact   -1.008071e+01
CO3-2.tot        1.696006e-11
CO3-2.con        7.399651e-15
CO3-2.logact    -1.413140e+01
CO3-2.diss       1.696006e-11
HCO3-.tot        1.411291e-11
HCO3-.con        1.411291e-11
HCO3-.logact    -1.085054e+01
HCO3-.diss       1.411291e-11
H2CO3.tot        2.839753e-12
H2C

In [5]:
# %%
# Scenario 2, fixed pressure
# InVars_pv need to contain floats
IN = np.array([np.ones_like(InVars_pv)]).astype(float)

IN[0][np.where(InVars_pv == 'gas_type')] = 1 # fixed gasvolume = 0, fixed pressure = 1
IN[0][np.where(InVars_pv == 'CO2[g].tot')] = 1e-10 # moles virtually no CO2 in system
IN[0][np.where(InVars_pv == 'Ar[g].tot')] = 4.1e-2 # moles for about 1 atm Argon pressure (estimated from ideal gas law)
IN[0][np.where(InVars_pv == 'watervolume')] = 1.0 # 1 liter
IN[0][np.where(InVars_pv == 'gasvolume_fixed')] = 1.0 # 1 liter
IN[0][np.where(InVars_pv == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_pressure_volume.set_and_calculate(IN)
Res_Scenario2 = pd.DataFrame(OUT,columns=OutVars_pv)
print(Res_Scenario2.T)

                            0
pH               7.048032e+00
H+.con           8.956139e-08
H+.tot           8.949203e-08
H+.logact       -7.048032e+00
OH-.tot          8.949203e-08
OH-.con          8.949203e-08
OH-.logact      -7.048368e+00
CO2[g].tot       1.000000e-10
CO2[g].con       4.078515e-10
CO2[g].diss      8.327809e-11
CO2[g].logact   -9.389498e+00
CO3-2.tot        8.327809e-11
CO3-2.con        3.632105e-14
CO3-2.logact    -1.344045e+01
CO3-2.diss       8.327809e-11
HCO3-.tot        6.929430e-11
HCO3-.con        6.929430e-11
HCO3-.logact    -1.015946e+01
HCO3-.diss       6.929430e-11
H2CO3.tot        1.394747e-11
H2CO3.con        1.394747e-11
H2CO3.logact    -1.085550e+01
H2CO3.diss       1.394747e-11
Ar[g].tot        4.100000e-02
Ar[g].con        1.000000e+00
Ar[g].diss       1.000000e-20
Ar[g].logact    -1.771683e-10
I                8.956142e-08
chargebalance    0.000000e+00
watervolume      1.000000e+00
gasvolume        4.100000e-02
pressure         1.000000e+00
gas_type  

In [6]:
# %%
# Scenario 3, fixed logactivity
# InVars_fl need to contain floats
IN = np.array([np.ones_like(InVars_fl)]).astype(float)

IN[0][np.where(InVars_fl == 'gas_type')] = 1 # fixed gasvolume = 0, fixed pressure = 1
IN[0][np.where(InVars_fl == 'CO2[g].logact')] = -10 # moles CO2-pressure = 420 ppm
IN[0][np.where(InVars_fl == 'Ar[g].logact')] = 1 # moles for about 1 atm Argon pressure (estimated from ideal gas law)
IN[0][np.where(InVars_fl == 'watervolume')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'gasvolume_fixed')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_fixed_logact.set_and_calculate(IN)
Res_Scenario3 = pd.DataFrame(OUT,columns=OutVars_fl)
print(Res_Scenario3.T)

Try a first calculation with iia switched off....
Parsing expressions of chemistry_fixed_logact.inp..... 
Optimizing expressions of chemistry_fixed_logact.inp..... 0.045 sec.
898 variables, 2117 expressions, 0 equations.
First calculation was successful!
Repeat calculation with iia switched on..
Switching on: logI: -2
Switching on: pH: 7
This was successful!!
                            0
pH               7.048159e+00
H+.con           8.953521e-08
H+.tot           8.951819e-08
H+.logact       -7.048159e+00
OH-.tot          8.951819e-08
OH-.con          8.951819e-08
OH-.logact      -7.048241e+00
CO2[g].tot       2.142370e-11
CO2[g].con       1.000000e-10
CO2[g].diss      2.042370e-11
CO2[g].logact   -1.000000e+01
CO3-2.tot        2.042370e-11
CO3-2.con        8.910667e-15
CO3-2.logact    -1.405070e+01
CO3-2.diss       2.042370e-11
HCO3-.tot        1.699505e-11
HCO3-.con        1.699505e-11
HCO3-.logact    -1.076983e+01
HCO3-.diss       1.699505e-11
H2CO3.tot        3.419743e-12
H2CO3.co

## Assignment
### Making Soda water in a sodastream machine
Soda water from a sodastream is made with a very simple recipe: Tap water is pressurized with pure CO2 under high pressure. The maximum pressure is regulated to 8 atm of CO2[g]. 

Sparkling water gets it's taste from the acidity originating from dissolving CO2[g] in water.

```{exercise}
**Calculate the pH of the soda stream solution in the closed bottle.**

You can use the codes above to estimate the pH of the solution in a closed Soda bottle.

Steps:
1. decide which scenario you need to start with? Think how the Sodastream adds the CO2 to the water. Using a search on internet you can find how SodaStream works.
2. copy the content of the required scenario to a fresh python cell in this notebook (which you should download to your own computer).
3. modify the script so that it matches your scenario
4. process the output.
```

In [7]:
# %%
# Scenario 3, fixed logactivity
# InVars_fl need to contain floats
IN = np.array([np.ones_like(InVars_fl)]).astype(float)

IN[0][np.where(InVars_fl == 'gas_type')] = 0 # fixed gasvolume = 0, fixed pressure = 1
IN[0][np.where(InVars_fl == 'CO2[g].logact')] = np.log10(8.0) # log10(8 atm)
IN[0][np.where(InVars_fl == 'Ar[g].logact')] = np.log10(1e-10) # negligible Argon pressure
IN[0][np.where(InVars_fl == 'watervolume')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'gasvolume_fixed')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_fixed_logact.set_and_calculate(IN)
Res_Scenario_Soda_in_bottle = pd.DataFrame(OUT,columns=OutVars_fl)
print(f' The pH of the water is {Res_Scenario_Soda_in_bottle['pH'].values[0]:.2f}.')
print(f' The total carbonate concentration in solution is {Res_Scenario_Soda_in_bottle['CO3-2.diss'].values[0]:.3f} moles/liter')


 The pH of the water is 3.46.
 The total carbonate concentration in solution is 0.274 moles/liter


### Why will Soda water taste flat after leaving it in a glass open to the atmosphere after some time?
When leaving a glass of soda water open to the atmosphere for some time, the taste will change and it will become much less sparkling: it will become "flat".

Use the correct scenario from above to calculate the composition of soda water after it is in equilibrium with the atmosphere?
Steps:
1. decide which scenario you need to start with? Think about the equilibrium state of water after it has been left open to that atmosphere for some time.
2. copy the content of the required scenario to a fresh python cell in this notebook (which you should download to your own computer).
3. modify the script so that it matches your scenario
4. process the output.


In [8]:
# %%
# Scenario 3, fixed logactivity
# InVars_fl need to contain floats
IN = np.array([np.ones_like(InVars_fl)]).astype(float)

IN[0][np.where(InVars_fl == 'gas_type')] = 0 # fixed gasvolume = 0, fixed pressure = 1
IN[0][np.where(InVars_fl == 'CO2[g].logact')] = np.log10(420e-6) # log10(8 atm)
IN[0][np.where(InVars_fl == 'Ar[g].logact')] = np.log10(1) # back ground pressure is 1 atm, we assume Ar represents the atmosphere!
IN[0][np.where(InVars_fl == 'watervolume')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'gasvolume_fixed')] = 1.0 # 1 liter
IN[0][np.where(InVars_fl == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_fixed_logact.set_and_calculate(IN)
Res_Scenario_Soda_open_glass = pd.DataFrame(OUT,columns=OutVars_fl)
print(f' The pH of the water is {Res_Scenario_Soda_open_glass['pH'].values[0]:.2f}.')
print(f' The total carbonate concentration in solution is {Res_Scenario_Soda_open_glass['CO3-2.diss'].values[0]:.3e} moles/liter')


 The pH of the water is 5.60.
 The total carbonate concentration in solution is 1.689e-05 moles/liter


### What is the volume of CO2[g] pumped in to 0.5 liter of Soda water by the sodastream machine?
Soda water is sparkling when we open the bottle, clear gas is leaving the water when we release the pressure. We can use one of the scenario's above to calculate the volume of gas added to the Soda water if it were released at 1 atm.

Steps:
1. decide which scenario you need to start with? We can develop a thought experiment to measure the volume of gas released at 1 atmosphere. One way to do this is to release the gas in to a very flexible balloon of which we can measure the volume. You can use one of the scenarios from above to solve this problem.
2. copy the content of the required scenario to a fresh python cell in this notebook (which you should download to your own computer).
3. modify the script so that it matches your scenario. Please note that for the initial condition you need to take the results from the scenario where you created the Soda water.
4. process the output.

In [11]:
# %%
# Scenario 2, fixed pressure
# InVars_pv need to contain floats
IN = np.array([np.ones_like(InVars_pv)]).astype(float)

IN[0][np.where(InVars_pv == 'gas_type')] = 1 # fixed gasvolume = 0, fixed pressure = 1
# The amount of CO2[g].tot comes from the Res_Scenario_Soda_in_bottle
# The total amount of CO2[g] in this last step is equal to the amount of CO3-2 dissolved in solution
IN[0][np.where(InVars_pv == 'CO2[g].tot')] = Res_Scenario_Soda_in_bottle['CO3-2.diss'].values[0] # 
# We have a negligible amount of Ar[g].tot in the system as pure CO2[g] was used in the sodastream machine.
IN[0][np.where(InVars_pv == 'Ar[g].tot')] = 1e-10 # moles for about 1 atm Argon pressure (estimated from ideal gas law)
# This scenario works with 0.5 liter of water
IN[0][np.where(InVars_pv == 'watervolume')] = 0.5 # 1 liter
# gasvolume_fixed will be neglected for this scenario as we aim to calculate the gasvolume
# IN[0][np.where(InVars_pv == 'gasvolume_fixed')] = 1.0 # 1 liter 
# The background pressure is set to 1 atm in 'pressure_fixed'
IN[0][np.where(InVars_pv == 'pressure_fixed')] = 1.0 # 1 liter  note: will not be used because of gas_type=0

# Calculate equilibrium conditions for initial situation 
OUT = p_pressure_volume.set_and_calculate(IN)
Res_Scenario_balloon = pd.DataFrame(OUT,columns=OutVars_pv)
print(f'The volume of CO2[g] released from 0.5 liter of sodastream water is {Res_Scenario_balloon['gasvolume'].values[0]:.3e} liter.')
print(f'The partial pressure of CO2[g] released from 0.5 liter of sodastream water is {Res_Scenario_balloon['CO2[g].con'].values[0]:.3e} atm.')

The volume of CO2[g] released from 0.5 liter of sodastream water is 2.568e-01 liter.
The partial pressure of CO2[g] released from 0.5 liter of sodastream water is 1.000e+00 atm.


```{exercise}
Check these results using your own calculations in python using the Ideal Gas Law and Henry's coefficient for CO2[g].
Do this calculations in a python script.
```